In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome,
                           perform_experiments, ema_logging,Policy, MultiprocessingEvaluator)

from ema_workbench import Samplers

from dike_model_function import DikeNetwork  # @UnresolvedImport

from problem_formulation import get_model_for_problem_formulation

from ema_workbench.em_framework.salib_samplers import get_SALib_problem

from SALib.analyze import sobol
import SALib

import pickle

In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO);

In [ ]:
dike_model, planning_steps = get_model_for_problem_formulation(2,True)

In [ ]:
planning_steps

In [ ]:
uncertainties = dike_model.uncertainties 
outcomes = dike_model.outcomes
problem = get_SALib_problem(uncertainties)
print(problem)

In [ ]:
#for unc in dike_model.uncertainties:
#    print(repr(unc))

In [ ]:
#for out in dike_model.outcomes:
#    print(repr(out))

In [ ]:
#for lev in dike_model.levers:
    print(repr(lev.name))

In [ ]:
def get_0_dict():
    return {l.name: 0 for l in dike_model.levers}

# Creates a Policy object from a dict and a (optional) name
def create_policy(dict1, name=None):
    return Policy(f"Policy_{name}", **dict(get_0_dict(), **dict1))

In [ ]:
pol = create_policy({'0_RfR 0': 0,
     '1_RfR 0': 0,
     '2_RfR 0': 0,
     '3_RfR 0': 0,
     '4_RfR 0': 0,
     'EWS_DaysToThreat': 0,
     'A.1_DikeIncrease 0': 0,
     'A.2_DikeIncrease 0': 0,
     'A.3_DikeIncrease 0': 0,
     'A.4_DikeIncrease 0': 0,
     'A.5_DikeIncrease 0': 0,},'Upstream dike, downstream rfr')

In [ ]:
pol

In [ ]:
policies = pol
sobol_switch = True
n_exp = 1000
   
use_pickle1 = False
if use_pickle1:
    with open('data/SOBOL_results1.pickle','rb') as filehandler:
        results = pickle.load(filehandler)
elif sobol_switch == True :
    with MultiprocessingEvaluator(dike_model,n_processes=10) as evaluator:
        results = evaluator.perform_experiments(scenarios=n_exp, uncertainty_sampling=Samplers.SOBOL)
    
    with open('data/SOBOL_results1.pickle',"wb") as filehandler:
        pickle.dump(results,filehandler)
else: 
    with MultiprocessingEvaluator(dike_model,n_processes=10) as evaluator:
        results = evaluator.perform_experiments(scenarios=n_exp, policies = pol, uncertainty_sampling=Samplers.SOBOL)
    
    with open('data/SOBOL_results1.pickle',"wb") as filehandler:
        pickle.dump(results,filehandler)  
experiments_sobol, outcomes_sobol = results

In [ ]:
deaths_sobol = sobol.analyze(problem, outcomes_sobol['Expected Number of Deaths'],calc_second_order=True, print_to_console=False)
damages_sobol = sobol.analyze(problem, outcomes_sobol['Expected Annual Damage'],calc_second_order=True, print_to_console=False)

In [ ]:
Si_filter = {k:deaths_sobol[k] for k in ['ST','ST_conf','S1','S1_conf']}
Si_df_deaths = pd.DataFrame(Si_filter, index=problem['names'])
Si_df_deaths['ST-S1'] = Si_df_deaths['ST'] - Si_df_deaths['S1']
Si_df_deaths['Second order interaction'] = np.where(Si_df_deaths['ST-S1'] > 0.05, True, False)
Si_df_deaths_2nd = Si_df_deaths.loc[Si_df_deaths['Second order interaction'] == True]
Si_df_deaths_2nd

In [ ]:
## However, the S2 dataframe shows these interactions are not very strong
#pd.set_option('display.max_columns', None)
#S2_deaths_sobol = pd.DataFrame(deaths_sobol['S2'])
#S2_deaths_sobol.fillna(0,inplace=True)
#S2_deaths_sobol
#S2_deaths_sobol_boolean = S2_deaths_sobol>0.05
#S2_deaths_sobol_boolean

In [ ]:
sns.set_style('white')
fig, ax = plt.subplots(1)

indices = Si_df_deaths[['S1','ST']]
indices = indices.loc[~(Si_df_deaths==0).all(axis=1)]
err = Si_df_deaths[['S1_conf','ST_conf']]
err = err.loc[~(Si_df_deaths==0).all(axis=1)]

indices.plot.bar(yerr=err.values.T,ax=ax)
fig.set_size_inches(10,8)
fig.subplots_adjust(bottom=0.3)
fig.savefig("sobol_deaths.png", dpi=200)
plt.show()

In [ ]:
Si_filter = {k:damages_sobol[k] for k in ['ST','ST_conf','S1','S1_conf']}
Si_df_damages = pd.DataFrame(Si_filter, index=problem['names'])
Si_df_damages['ST-S1'] = Si_df_damages['ST'] - Si_df_damages['S1']
Si_df_damages['Second order interaction'] = np.where(Si_df_damages['ST-S1'] > 0.05, True, False)
Si_df_damages_2nd = Si_df_damages.loc[Si_df_damages['Second order interaction'] == True]
Si_df_damages_2nd

In [ ]:
#S2_damages_sobol = pd.DataFrame(damages_sobol['S2'])
#S2_damages_sobol.fillna(0,inplace=True)
#S2_damages_sobol
#S2_damages_sobol_boolean = S2_damages_sobol>0.05
#S2_damages_sobol_boolean

In [ ]:
sns.set_style('white')
fig, ax = plt.subplots(1)

indices = Si_df_damages[['S1','ST']]
indices = indices.loc[~(Si_df_damages==0).all(axis=1)]
err = Si_df_damages[['S1_conf','ST_conf']]
err = err.loc[~(Si_df_damages==0).all(axis=1)]

indices.plot.bar(yerr=err.values.T,ax=ax)
fig.set_size_inches(10,8)
fig.subplots_adjust(bottom=0.3)
fig.savefig("sobol_damages.png", dpi=200)
plt.show()